In [1]:
import os
import numpy as np
import cv2
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:

# ----------------------------------------load data module begin-----------------------------------------------------
def load_path(data_set, root_dir = 'B:/MURA/MURA-v1.1/', data_type = 'XR_ELBOW'):
    # load MURA data and return data path list and label numpy array
    data_path = root_dir + data_set + '/' + data_type + '/'
    Path = []
    labels = []
    for root, dirs, files in os.walk(data_path):  # read all images, os.walk returns iterators genertor traverses all files
        for name in files:
            if root.split('_')[-1] == 'positive':  # positive label == 1；
                path_1 = os.path.join(root, name)
                Path.append(path_1)
                labels += [1]
            elif root.split('_')[-1] == 'negative':  # negative label == 1；
                path_1 = os.path.join(root, name)
                Path.append(path_1)
                labels += [0]
            else:
                continue
    labels = np.asarray(labels)
    return Path, labels

#  path list in, image list out
def load_image(Path='../valid', imsize=224):
    Images = []
    i = 1
    for path in Path:

        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print("null")
            continue
        else:
            if i % 500 == 0:
                print ("loading image ", i)
            
            image = cv2.resize(image, (imsize, imsize))   # resize images
            image = randome_rotation_flip(image, imsize)  # predeal images
            Images.append(image)
            i = i+1

    Images = np.asarray(Images).astype('float32')

    # normalization
    print("Starting normalization.......")
    mean = np.mean(Images[:, :, :])
    std = np.std(Images[:, :, :])
    Images[:, :, :] = (Images[:, :, :] - mean) / std
    print("normalization finished")

    Images = np.expand_dims(Images, axis=1)

    return Images

# inner function used by load_image
def randome_rotation_flip(image, imsize=224):
    if random.randint(0, 1):
        image = cv2.flip(image, 1)

    if random.randint(0, 1):
        angle = random.randint(-30, 30)
        M = cv2.getRotationMatrix2D((imsize / 2, imsize / 2), angle, 1)
        # third parameter: the transformed image size
        image = cv2.warpAffine(image, M, (imsize, imsize))
    return image

# ----------------------------------------load data module end-----------------------------------------------------


In [3]:
# -----------------------------------network hyperparameter begin--------------------------------
im_size = 224  # input image size
batch_size = 50 
epochs = 20
x_shape  = (batch_size, 1, 224, 224)
model_save = True
model_reload = False
drop_out_rate = 0.2
my_init = keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

# -----------------------------------network hyperparameter end--------------------------------


In [4]:
# ----------------------------------------net build module begin-----------------------------------------------------

if model_reload:
    deep_model = keras.models.load_model('MURA_keras_7_21.h5')
else:
    deep_model = keras.Sequential(
    [
    layers.BatchNormalization(input_shape=((x_shape[1], x_shape[2], x_shape[3])),axis=1),
    
    #1
    layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1),padding='same', 
                  kernel_initializer=my_init, 
                  activation='selu', data_format='channels_first'),
    layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1),padding='same', 
                  kernel_initializer=my_init, 
                  activation='selu', data_format='channels_first'),
    layers.MaxPool2D(pool_size=(2,2), data_format='channels_first'),
    
    layers.BatchNormalization(axis=1),
    
    #2
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1),padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.MaxPool2D(pool_size=(2,2), data_format='channels_first'),
    
    layers.BatchNormalization(axis=1),
    
    #3
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.MaxPool2D(pool_size=(2,2), data_format='channels_first'),
    
    layers.BatchNormalization(axis=1),
    
    #4
    layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.MaxPool2D(pool_size=(2,2), data_format='channels_first'),
    layers.Dropout(drop_out_rate),
    
    layers.BatchNormalization(axis=1),
    
    #5
    layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1),padding='same', 
                      kernel_initializer=my_init, 
                      activation='selu', data_format='channels_first'),
    layers.MaxPool2D(pool_size=(7,7), data_format='channels_first'),
    
    #6
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dropout(drop_out_rate),
    layers.Dense(10, activation='selu', kernel_initializer=my_init),
        
    #7
    layers.BatchNormalization(),
    layers.Dropout(drop_out_rate),
    layers.Dense(1, activation='sigmoid', kernel_initializer=my_init),
     ])
    
    deep_model.compile(optimizer=keras.optimizers.Adam(),
                 loss='binary_crossentropy',
                metrics=['accuracy'])
    deep_model.summary()
        
# ----------------------------------------net build module begin-----------------------------------------------------

        

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v2 (Batc (None, 1, 224, 224)       4         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 224, 224)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 224, 224)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 112, 112)      0         
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 16, 112, 112)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 112, 112)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 112, 112)      9

In [5]:
# -------------------------------load train data module begin-----------------------------

# 训练数据
im_size = 224
x_train_path, y_train = load_path(data_set='train')
x_train = load_image(x_train_path, imsize=im_size)
print(x_train.shape)
print(y_train.shape)
  
# -------------------------------load train data module end-----------------------------


loading image  500
loading image  1000
loading image  1500
loading image  2000
loading image  2500
loading image  3000
loading image  3500
loading image  4000
loading image  4500
Starting normalization.......
normalization finished
(4931, 1, 224, 224)
(4931,)


In [11]:
# 开始训练
history = deep_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)   

Train on 4437 samples, validate on 494 samples
Epoch 1/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6564 - accuracy: 0.6137 - val_loss: 0.4820 - val_accuracy: 0.9534
Epoch 2/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6518 - accuracy: 0.6223 - val_loss: 0.5364 - val_accuracy: 0.8806
Epoch 3/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6613 - accuracy: 0.6121 - val_loss: 0.4996 - val_accuracy: 0.9150
Epoch 4/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6517 - accuracy: 0.6166 - val_loss: 0.4711 - val_accuracy: 0.9534
Epoch 5/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6362 - accuracy: 0.6446 - val_loss: 0.4570 - val_accuracy: 0.9636
Epoch 6/10
4437/4437 [==============================] - 38s 9ms/sample - loss: 0.6330 - accuracy: 0.6374 - val_loss: 0.4423 - val_accuracy: 0.9777
Epoch 7/10
4437/4437 [==============================] - 38s 9ms/sample 

In [14]:
# test
x_test_path, y_test = load_path(data_set='valid')
x_test = load_image(x_test_path, im_size)
res = deep_model.evaluate(x_test, y_test)  

Starting normalization.......
normalization finished
465/465 [==============================] - 1s 3ms/sample - loss: 0.6512 - accuracy: 0.6172


In [10]:
if model_save:
    deep_model.save('MURA_keras_7_21.h5')